- [Paper: Optimal Online Adaptive Electric Vehicle Charging](http://netlab.caltech.edu/assets/publications/Guo-2017-OLP.pdf)
- [Pyomo - Optimization Modeling in Python](https://pyomo.readthedocs.io/en/stable/)
- [ TeX functions supported by KaTeX](https://katex.org/docs/supported.html)
- [TGC_LP: OLP in Excel for Enexis (OneDrive)](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

# OLP: Online Linear Programming

<br>

<div style="border:1px solid blue; border-radius: 5px; background-color: #DDEDF9; color: black; padding: 0px">
    <div style="background-color: darkblue; color: white; padding: 10px; border-radius: 5px 5px 0 0">
        <strong>‼ Note:</strong>
    </div>
    <BR>
    >  The outcomes of the excel-model are the same as the pyomo-model !!! 
    <BR>
    >  Check: <strong>objective outcome</strong> and the <strong>Power</strong> (Variables) 
    <BR>
    <BR>
</div>

### ✔ Pyomo-0: optimization of energy charged for single EV 

#### Model

This first model has only 1 EV/SE and has to determine the ``kW`` for the next ``n`` periods (n=5).  

The objective is to minimize the difference in the  ``theoretical maximum energy output [kWh]``, being ```Enexis max power [kW] * duration [h]``` and the ``actual delivered energy``, being ```actual power [kW] * duration [h]```.

By normalizing the duration, the resulting ``weighted-% energy delivered`` can easily be combined with any future objective like the ``weighted-% customer satisfaction``. 

This results in the following ``Objective function`` to be minimalized. 


##### Objective Function

$$
\begin{align*}
\text{minimize} \; & \left( 
    \alpha \sum_{j=1}^{n} w_{j} \left( 1 - \frac{x_{j} \; t_{j}}{ex_{j} \; t_{j} } \right)  
\right)\\\\
\text{where} \; & w_{j} = \frac{t_{j}}{\sum_{k=1}^{n} t_{k}}, \quad \\
& \forall j,k \in \{1, 2, \ldots, n\}
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.} \\\\
& w_{j} \text{ represents a weight factor, to give greater emphasis to longer time periods} \\
& x_{j} \text{ represents the power [kW] in time period } j. \\
& ex_{j} \text{ represents the maximum power available [kW] in time period } j. \\
& t_{j} \text{ represents the duration [h] of time period } j. \\\\
& \alpha \text{ importance to optimize utilization} \\
\end{array} 
$$

##### Decision Variables

The charging power at time slot $t$ are the decision variables, $x_{t}$ 

##### Constraints

The charging power to be used ($x_{t}$) in kW is subject to 4 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output [kW] ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input [kW] ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output [kW] ($se_j$) of the EVSE.


$$
\begin{array}{llllll}
\text{s.t.} & x_{j} \geq 0  &  \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq ex_{j} & \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq ev_{j} & \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq se_{j} & \forall j \in \{1, 2, \ldots, n\} \\
\end{array} 
$$

$ \text{Note: } j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}$ 

This example relates to the ``pyomo_0`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_0``) 

<img src="./images/pyomo_0.png" width="1000">

#### Code & Results

##### Session: EV/EVSE

In [1]:
from pyomo.environ import *
import numpy as np

# tested with:
# solver = "ipopt" #  m 50 EVSE's, > 50 time periods
# solver = "mosek" #  m 50 EVSE's, > 50 time periods
# solver = "cplex"  #   m 50 EVSE's, 18 time periods
solver = "glpk" #   m 50 EVSE's, > 50 time periods
# solver = "gurobi" # m xx EVSE's, 19 time periods - no license yet

# row reserved for future use
n = 5  # number of time periods in the future

# --------------------------------------------------------------------------
# Data from each Session EV & SE

alpha = 1.0  # EVSE efficiency


EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period
EX_MPO = 14.72  # enexis max Power output for each time period (constant)


EV01 = np.array([7.36, 7.36, 5.00, 3.68, 0.00])
SE01 = np.array([5.00, 7.36, 5.00, 3.68, 0.00])
ES01 = np.minimum(EV01, SE01)

EVSE = np.array([ES01])

##### TGC: Central Controller

In [2]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

# Create the weights w[j] for the parking
pt = np.array([40, 10, 20, 10, 10])
w = pt / np.sum(pt)

# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()


# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (j + 1): EVSE[i][j]
    for i in range(len(EVSE))
    for j in range(len(EVSE[i]))
}
model.session = Param(model.J, initialize=session_max_kw)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------

def obj_expression(model):
    # return the expression for the objective
    return alpha * sum(w[j - 1] * (1 - model.x[j] / model.enexis[j]) for j in model.J)



model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power <= enexis max power output for period j?
def demand_constraint_rule(model, j):
    return model.x[j] <= model.enexis[j]


# is ev/se power <= ev max power input for period j?
def session_constraint_rule(model, j):
    return model.x[j] <= model.session[j]


# the next lines create one constraint for each member of the set model.J
model.Enexis__Constraint = Constraint(model.J, rule=demand_constraint_rule)
model.Session_Constraint = Constraint(model.J, rule=session_constraint_rule)


# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory(solver) 

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()
print("\n")
print(f"EVSE 01: FC: {round(sum(pt[x-1] * tgc.x[x].value for x in range(1, n+1)), 2)} kWh\tPwr: {[round(tgc.x[x].value, 2) for x in range(1, n+1)]} ")


print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))





EVSE 01: FC: 410.4 kWh	Pwr: [5.0, 7.36, 5.0, 3.68, 0.0] 

Objective Outcome =  0.6902


### ✔ Pyomo-1: optimization of energy charged for multiple EVs 

#### Model

Next, we extend the ``pyomo-0`` model by allowing multiple EV/SE parallel charging sessions, (m = 3).  
This results in the following ``Objective function`` to be minimized.

##### Objective Function

$$
\begin{align*}
\text{minimize} \; & \left( 
    \alpha \sum_{j=1}^{n} w_{j} \left( 1 - \frac{e_{j}}{E_{j}} \right)  
\right)\\\\
\text{where} \; & w_{j} = \frac{t_{j}}{\sum_{k=1}^{n} t_{k}}, \quad \\
& e_{j} = \sum_{i=1}^{m} x_{ij} \; t_{j}, \quad E_{j} = ex_{j} \; t_{j}, \\
& \forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& i \text{ represents a specific charge session EV/SE with } m \text{ being the maximum parallel charge sessions possible.} \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.} \\\\
& w_{j} \text{ represents a weight factor, to give greater emphasis to longer time periods} \\
& e_{j} \text{ represents the total energy [kWh] charged in time period } j. \\
& E_{j} \text{ represents the maximum energy available [kWh] in time period } j. \\
& t_{j} \text{ represents the duration [h] of time period } j. \\\\
& \alpha \text{ importance to optimize utilization} \\
\end{array} 
$$


##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($ x_{ij} $) in kW is subject to 4 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.


$$
\begin{array}{llll}
\text{s.t.} & x_{ij} \geq 0  &  \\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} & \\
& x_{ij} \leq ev_{ij} & \\
& x_{ij} \leq se_{ij} & \\ 
\end{array} 
$$



$$
\begin{array}{lll}
\forall i \in \{1, 2, \ldots, m\}, &\text{i =  charge session,} & \text{m = 3}  \\
\forall j \in \{1, 2, \ldots, n\}, &\text{j =  time period,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_1``) 


<img src="./images/pyomo_1.png" width="1000">

#### Code & Results

##### Sessions: EV/EVSE

In [3]:
from pyomo.environ import *
import numpy as np

# tested with:
# solver = "ipopt" #  m 50 EVSE's, > 50 time periods
# solver = "mosek" #  m 50 EVSE's, > 50 time periods
# solver = "cplex"  #   m 50 EVSE's, 18 time periods
solver = "glpk" #   m 50 EVSE's, > 50 time periods
# solver = "gurobi" # m xx EVSE's, 19 time periods - no license yet

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

alpha = 1.0  # EVSE efficiency

# --------------------------------------------------------------------------
# Data from each Session EV & SE

EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period
EX_MPO = 14.72  # enexis max Power output for each time period (constant)

# For each Session EV & SE Compare the arrays and keep the minimum
EV01 = np.array([7.36, 7.36, 5.00, 3.68, 0.00])
SE01 = np.repeat(SE_MPO, n)
SE01 = np.array([5.00, 7.36, 5.00, 3.68, 0.00])
ES01 = np.minimum(EV01, SE01)

EV02 = np.array([7.36, 7.36, 5.00, 3.68, 0.00])
SE02 = np.array([5.00, 7.36, 5.00, 3.68, 0.00])
ES02 = np.minimum(EV02, SE02)

EV03 = np.array([7.36, 7.36, 5.00, 3.68, 0.00])
SE03 = np.array([5.00, 7.36, 5.00, 3.68, 0.00])
ES03 = np.minimum(EV03, SE03)

EVSE = np.array([ES01, ES02, ES03])


##### TGC: Central Controller

In [4]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

# Create the weights w[j] for the parking 
pt = np.array([40, 10, 20, 10, 10])
w = pt/np.sum(pt)

# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j]
    for i in range(len(EVSE))
    for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return alpha * sum(w[j-1] * (1 - sum(model.x[i, j] for i in model.I)/model.enexis[j]) 
               for j in model.J)

model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def demand_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= ev max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# the next line creates one constraint for each member of the set model.J
model.Enexis__Constraintt = Constraint(model.J, rule=demand_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)


# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory(solver) 

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()

# print the power per EVSE per time period
def EVSE_print():
    for i in range(1, m + 1):
        print(
            f"EVSE {str(i).zfill(2)}: FC: {round(sum(pt[x-1] * tgc.x[i, x].value for x in range(1, n+1)), 2)}    Pwr: {[round(tgc.x[i, x].value, 2) for x in range(1, n+1)]} kW"
        )


EVSE_print()

# print the objective outcome
print("\n")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))

EVSE 01: FC: 410.4    Pwr: [5.0, 7.36, 5.0, 3.68, 0.0] kW
EVSE 02: FC: 410.4    Pwr: [5.0, 7.36, 5.0, 3.68, 0.0] kW
EVSE 03: FC: 320.0    Pwr: [4.72, 0.0, 4.72, 3.68, 0.0] kW



Objective Outcome =  0.1389


### ✔ Pyomo-2: optimization Pyomo-1 including Customer Satisfaction 

<br>

<div style="border:1px solid blue; border-radius: 5px; background-color: #DDEDF9; color: black; padding: 0px">
    <div style="background-color: darkblue; color: white; padding: 10px; border-radius: 5px 5px 0 0">
        <strong>‼ Note:</strong>
    </div>
    <BR>
    >  The <strong>objective outcome</strong> of the excel-model is the same as the pyomo-model !!! 
    <BR>
    >  The <strong>decision variable</strong> are different though!! (multiple solutions seem to be possible) 
    <BR>
    <BR>
</div>

#### Model

The Enexis Grid Connection (ECG) utilization (%) is the highest priority, as modelled in ``pyomo-1``.
 
Next, the customer satisfaction is taken into account as second priority. This will be measured as % of ``desired charge`` $dc_{i}$ which has been delivered at the time the customer leaves. The ``final charge`` $fc_{i}$ in kWh can be calculated when we know for each EV the parking time ($pt_{i}$).

$$
fc_{i} = \sum_{j=1}^{n} x_{ij} \; t_{j}
$$

Please note that the model shouldn't be penalized for unrealistic customer demands. The customers ``desired charge`` $dc_{i}$ in (kWh) may never be higher than the ``realistc charge`` $rc_{i}$, being the maximum energy which can be deliverd at maximum charging power during the time of parking!  

$$
rc_{i} = \sum_{j=1}^{n} \min(ev_{ij}, se_{ij}) \; t_{j}
$$

The customer satisfaction will be expressed as: 

$$
\text{customer satisfaction}_{i} =  \frac{fc_{i}}{rc_{i}} \; \\\\
$$

$$ \forall i \in \{1, 2, \ldots, m\}, \; \forall j \in \{1, 2, \ldots, n\} $$

##### Objective Function

Modifying the ``pyomo-1`` model results in the following ``Objective function`` to be minimized:  
<br>
<br>

$$
\begin{align*}
\text{minimize} \; & \left( 
    \alpha \sum_{j=1}^{n} w_{j} \left( 1 - \frac{e_{j}}{E_{j}} \right) + 
    \frac{\beta}{m} \sum_{i=1}^{m} \left(1 - \frac{fc_{i}}{rc_{i}}  \right)  
\right)\\\\
\text{where} \; & w_{j} = \frac{t_{j}}{\sum_{k=1}^{n} t_{k}}, \quad \\
& e_{j} = \sum_{i=1}^{m} x_{ij} \; t_{j}, \quad E_{j} = ex_{j} \; t_{j}, \\
& fc_{i} = \sum_{j=1}^{n} x_{ij} \; t_{j}, \quad rc_{i} = \sum_{j=1}^{n} \min(ev_{ij}, se_{ij}) \; t_{j}, \\\\
& \forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& i \text{ represents a specific charge session EV/SE with } m \text{ being the maximum parallel charge sessions possible.} \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.} \\\\
& w_{j} \text{ represents a weight factor, to give greater emphasis to longer time periods} \\
& e_{j} \text{ represents the total energy [kWh] charged in time period } j. \\
& E_{j} \text{ represents the maximum energy available [kWh] in time period } j. \\
& t_{j} \text{ represents the duration [h] of time period } j. \\\\
& fc_{i} = \text{ represents the final charge [kWh] when customer leaves } \\
& rc_{i} = \text{ represents the realistic charge [kWh] being less or equal to desired charge} \\\\
& \alpha \text{ importance to optimize utilization} \\
& \beta \text{ importance to optimize desired customer demand} 
\end{array} 
$$

##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($x_{ij}$) in kW is subject to 5 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.
1. the ``final charge`` $fc_{i}$ of EV $i$  should be less or equal to the ``realistic charge`` $rc_{i}$  
(which is less or equal to the ``desired charge`` $dc_{i}$)


$$
\begin{array}{rllll}
\text{s.t.} & x_{ij} \geq 0  &\\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} &\\
& x_{ij} \leq ev_{ij} &\\
& x_{ij} \leq se_{ij} &\\
\sum_{j=1}^{n} & pt_{j} x_{ij} \leq rc_{i} & 
\end{array} 
$$



$$
\begin{array}{lll}
\forall i \in \{1, 2, \ldots, m\}, &\text{i =  charge session,} & \text{m = 3}  \\
\forall j \in \{1, 2, \ldots, n\}, &\text{j =  time period,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_2``) 

<img src="./images/pyomo_2.png" width="1000">

#### Code & Results

##### Sessions: EV/EVSE

In [5]:
from pyomo.environ import *
import numpy as np

# tested with:
# solver = "ipopt" #  m 50 EVSE's, > 50 time periods
# solver = "mosek" #  m 50 EVSE's, > 50 time periods
# solver = "cplex"  #   m 50 EVSE's, 18 time periods
solver = "glpk" #   m 50 EVSE's, > 50 time periods
# solver = "gurobi" # m xx EVSE's, 19 time periods - no license yet

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

alpha = 1.0  # EVSE efficiency
beta = 1.0  # customer satisfaction


# --------------------------------------------------------------------------
# Data from each Session EV & SE

EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period
EX_MPO = 14.72  # enexis max Power output for each time period (constant)

# For each Session EV & SE Compare the arrays and keep the minimum
EV01 = np.array([3.68, 7.36, 7.36, 3.68, 0.00])
SE01 = np.repeat(SE_MPO, n)
SE01 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES01 = np.minimum(EV01, SE01)
# print(ES01)

EV02 = np.array([7.36, 7.36, 7.36, 3.68, 3.68])
SE02 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES02 = np.minimum(EV02, SE02)
# print(ES02)

EV03 = np.array([7.36, 7.36, 7.36, 7.36, 3.68])
SE03 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES03 = np.minimum(EV03, SE03)
# print(ES03)

EVSE = np.array([ES01, ES02, ES03])
# print(EVSE)

##### TGC: Central Controller

In [6]:
# Create the weights w[j] for the parking 
pt = np.array([40, 10, 20, 10, 10])
w = pt/np.sum(pt)

# calclate the maximum charge possible for each session
session_max_charge = np.sum(pt * EVSE, axis=1)

# desired charge for each session
desired_charge = np.array([350, 200, 350])

# realistic charge for each session
realistic_charge = np.minimum(session_max_charge, desired_charge)


In [7]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j] for i in range(len(EVSE)) for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)

# realistic charge for each session
realistic_charge_kwh = {
    (i + 1): realistic_charge[i] for i in range(len(realistic_charge))
}
model.realistic_charge = Param(model.I, initialize=realistic_charge_kwh)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return alpha * sum(
        w[j - 1] * (1 - sum(model.x[i, j] for i in model.I) / model.enexis[j])
        for j in model.J
    ) + beta * sum(
        (
            1
            - sum(pt[j - 1] * model.x[i, j] for j in model.J)
            / model.realistic_charge[i]
        )
        for i in model.I
    ) / m


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def ex_grid_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= session max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# is final charge <= realistic charge <= desired charge
def deschrg_constraint_rule(model, i):
    return sum(pt[j - 1] * model.x[i, j] for j in model.J) <= model.realistic_charge[i]


# the next line creates one constraint for each member of the set model.J
model.Ex_Grid_Constraint = Constraint(model.J, rule=ex_grid_constraint_rule)
model.DesChrg_Constraint = Constraint(model.I, rule=deschrg_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)

# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory(solver)

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()

# print the power per EVSE per time period
def EVSE_print():
    for i in range(1, m + 1):
        print(
            f"EVSE {str(i).zfill(2)}: DC: {round(desired_charge[i-1], 2)} kWh\tRC: {round(realistic_charge[i-1], 2)} kWh\tFC: {round(sum(pt[x-1] * tgc.x[i, x].value for x in range(1, n+1)), 2)} kWh\tPwr: {[round(tgc.x[i, x].value, 2) for x in range(1, n+1)]} kW"
        )


EVSE_print()

# print the objective outcome
print("\n")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))

EVSE 01: DC: 350 kWh	RC: 350.0 kWh	FC: 350.0 kWh	Pwr: [3.68, 1.88, 7.36, 3.68, 0.0] kW
EVSE 02: DC: 200 kWh	RC: 200.0 kWh	FC: 200.0 kWh	Pwr: [5.0, 0.0, 0.0, 0.0, 0.0] kW
EVSE 03: DC: 350 kWh	RC: 350.0 kWh	FC: 350.0 kWh	Pwr: [5.0, 0.0, 7.36, 0.28, 0.0] kW



Objective Outcome =  0.3207


### Pyomo-3: Optimization Pyomo-2 including energy cost

#### Model

##### Dynamic pricing

<img src="./images/anwb.png" width="1000">

Finally we will extend the ``pyomo-2`` model to also take the energy cost 💰 per period into account



##### Objective Function

Modifying the ``pyomo-2`` model results in the following ``Objective function`` to be minimized:  
<br>
<br>

$$
\begin{align*}
\text{minimize} \; & \left( 
    \alpha \sum_{j=1}^{n} w_{j} \left( 1 - \frac{e_{j}}{E_{j}} \right) + 
    \frac{\beta}{m} \sum_{i=1}^{m} \left(1 - \frac{fc_{i}}{rc_{i}}  \right)  + 
    \frac{\gamma}{M_c} \sum_{j=1}^{n} c_{j} \; e_{j}  
\right)\\\\
\text{where} \; & w_{j} = \frac{t_{j}}{\sum_{k=1}^{n} t_{k}}, \quad \\
& e_{j} = \sum_{i=1}^{m} x_{ij} \; t_{j}, \quad E_{j} = ex_{j} \; t_{j}, \quad {M_c} = \sum_{j=1}^{n} E_{j}, \quad \\
& fc_{i} = \sum_{j=1}^{n} x_{ij} \; t_{j}, \quad rc_{i} = \sum_{j=1}^{n} \min(ev_{ij}, se_{ij}) \; t_{j}, \\\\
& \forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& i \text{ represents a specific charge session EV/SE with } m \text{ being the maximum parallel charge sessions possible.} \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.} \\\\
& w_{j} \text{ represents a weight factor, to give greater emphasis to longer time periods} \\
& e_{j} \text{ represents the total energy [kWh] charged in time period } j. \\
& E_{j} \text{ represents the maximum energy available [kWh] in time period } j. \\
& t_{j} \text{ represents the duration [h] of time period } j. \\\\
& {M_c} \text{ Maximum energy cost [€/kWh]} \\\\
& fc_{i} = \text{ represents the final charge [kWh] when customer leaves } \\
& rc_{i} = \text{ represents the realistic charge [kWh] being less or equal to desired charge} \\\\
& \alpha \text{ importance to optimize utilization} \\
& \beta \text{ importance to optimize desired customer demand} \\
& \gamma \text{ importance to optimize energy cost}
\end{array} 
$$

##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($x_{ij}$) in kW is subject to 5 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.
1. the ``final charge`` $fc_{i}$ of EV $i$  should be less or equal to the ``realistic charge`` $rc_{i}$  
(which is less or equal to the ``desired charge`` $dc_{i}$)


$$
\begin{array}{rllll}
\text{s.t.} & x_{ij} \geq 0  &\\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} &\\
& x_{ij} \leq ev_{ij} &\\
& x_{ij} \leq se_{ij} &\\
\sum_{j=1}^{n} & pt_{j} x_{ij} \leq rc_{i} & 
\end{array} 
$$



$$
\begin{array}{lll}
\forall i \in \{1, 2, \ldots, m\}, &\text{i =  charge session,} & \text{m = 3}  \\
\forall j \in \{1, 2, \ldots, n\}, &\text{j =  time period,} & \text{n = 5}
\end{array}
$$


This example relates to the ``pyomo_2`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_3``) 

<img src="./images/pyomo_3.png" width="1000">

#### Code & Results

##### Sessions: EV/EVSE

In [8]:
from pyomo.environ import *
import numpy as np

# tested with:
# solver = "ipopt" #  m 50 EVSE's, > 50 time periods
# solver = "mosek" #  m 50 EVSE's, > 50 time periods
# solver = "cplex"  #   m 50 EVSE's, 18 time periods
solver = "glpk" #   m 50 EVSE's, > 50 time periods
# solver = "gurobi" # m xx EVSE's, 19 time periods - no license yet

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

alpha = 1.0  # EVSE efficiency
beta = 1.0  # customer satisfaction
gamma = 1.0  # cost of energy

# --------------------------------------------------------------------------
# Data from each Session EV & SE

EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period
EX_MPO = 14.72  # enexis max Power output for each time period (constant)

# For each Session EV & SE Compare the arrays and keep the minimum
EV01 = np.array([3.68, 7.36, 7.36, 3.68, 0.00])
SE01 = np.repeat(SE_MPO, n)
SE01 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES01 = np.minimum(EV01, SE01)
# print(ES01)

EV02 = np.array([7.36, 7.36, 7.36, 3.68, 3.68])
SE02 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES02 = np.minimum(EV02, SE02)
# print(ES02)

EV03 = np.array([7.36, 7.36, 7.36, 7.36, 3.68])
SE03 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES03 = np.minimum(EV03, SE03)
# print(ES03)

EVSE = np.array([ES01, ES02, ES03])
# print(EVSE)

##### TGC: Central Controller

In [9]:
# Create the weights w[j] for the parking 
pt = np.array([40, 10, 20, 10, 10])
w = pt/np.sum(pt)

# calclate the maximum charge possible for each session
session_max_charge = np.sum(pt * EVSE, axis=1)

# desired charge for each session
desired_charge = np.array([350, 200, 350])

# realistic charge for each session
realistic_charge = np.minimum(session_max_charge, desired_charge)

# erngy price in euro/kWh for each time period
EnergyPrice = np.array([0.04, 0.08, 0.31, 0.28, 0.12])
Energy_Cost = EnergyPrice * pt



In [10]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# energy price per kWh for each time period
energy_price = {
    (j + 1): EnergyPrice[j]  for j in range(len(EnergyPrice))
}  
model.energy_price = Param(model.J, initialize=energy_price)

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j] for i in range(len(EVSE)) for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)

# realistic charge for each session
realistic_charge_kwh = {
    (i + 1): realistic_charge[i] for i in range(len(realistic_charge))
}
model.realistic_charge = Param(model.I, initialize=realistic_charge_kwh)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return (
        alpha
        * sum(
            w[j - 1] * (1 - sum(model.x[i, j] for i in model.I) / model.enexis[j])
            for j in model.J
        )
        + beta
        * sum(
            (
                1
                - sum(pt[j - 1] * model.x[i, j] for j in model.J)
                / model.realistic_charge[i]
            )
            for i in model.I
        )
        / m
        + (gamma/ sum(Energy_Cost[j-1] * model.enexis[j]  for j in model.J))
        * sum(
            (Energy_Cost[j-1] * sum(model.x[i, j] for i in model.I) )
            for j in model.J
        )
    )


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def ex_grid_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= session max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# is final charge <= realistic charge <= desired charge
def deschrg_constraint_rule(model, i):
    return sum(pt[j - 1] * model.x[i, j] for j in model.J) <= model.realistic_charge[i]


# the next line creates one constraint for each member of the set model.J
model.Ex_Grid_Constraint = Constraint(model.J, rule=ex_grid_constraint_rule)
model.DesChrg_Constraint = Constraint(model.I, rule=deschrg_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)

# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory(solver)

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()

# print the power per EVSE per time period
def EVSE_print():
    for i in range(1, m + 1):
        print(
            f"EVSE {str(i).zfill(2)}: DC: {round(desired_charge[i-1], 2)} kWh\tRC: {round(realistic_charge[i-1], 2)} kWh\tFC: {round(sum(pt[x-1] * tgc.x[i, x].value for x in range(1, n+1)), 2)} kWh\tPwr: {[round(tgc.x[i, x].value, 2) for x in range(1, n+1)]} kW"
        )


EVSE_print()

# print the objective outcome
print("\n")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))

EVSE 01: DC: 350 kWh	RC: 350.0 kWh	FC: 350.0 kWh	Pwr: [3.68, 7.36, 4.62, 3.68, 0.0] kW
EVSE 02: DC: 200 kWh	RC: 200.0 kWh	FC: 200.0 kWh	Pwr: [5.0, 0.0, 0.0, 0.0, 0.0] kW
EVSE 03: DC: 350 kWh	RC: 350.0 kWh	FC: 350.0 kWh	Pwr: [5.0, 7.36, 0.0, 3.96, 3.68] kW



Objective Outcome =  0.7957


### Pyomo-4: Optimization Pyomo-3 with m parallel charging sessions and horizon is n

#### Model

The ``pyomo-4`` model is the same as ``pyomo-3`` except for the number of parallel charging sessions m, which is increased from 3 to 20.

$$
\begin{array}{lll}
\forall i \in \{1, 2, \ldots, m\}, &\text{i =  charge session,} & \text{m = 20}  \\
\forall j \in \{1, 2, \ldots, n\}, &\text{j =  time period,} & \text{n = 10}
\end{array}
$$

#### Excel: results (sheet ``pyomo_4``) 


The excel solver is restricted to <span style="color:red">200 constraints.</span> and can't cope with this model

<img src="./images/pyomo_4.png" width="1000">

#### Code & Results

##### Sessions: EV/EVSE

In [11]:
from pyomo.environ import *
import numpy as np

print_solver_outcome = False
print_EVSE_power = False

print_session_max_charge = True
print_desired_charge = True
print_realistic_charge = True

print_parking_time = True
print_energy_price = False

# tested with:
# solver = "ipopt" #  m 50 EVSE's, > 50 time periods
# solver = "mosek" #  m 50 EVSE's, > 50 time periods
# solver = "cplex"  #   m 50 EVSE's, 18 time periods
solver = "glpk" #   m 50 EVSE's, > 50 time periods
# solver = "gurobi" # m xx EVSE's, 19 time periods - no license yet

m = 20  #  number of EVSEs
n = 10  #  number of time periods in the future
r = 0.5  # reduced Enexis grid connection

alpha = 1.0  # EVSE efficiency
beta = 1.0  #  customer satisfaction
gamma = 1.0  # cost of energy

# --------------------------------------------------------------------------
# Data from each Session EV & SE

# default values for EV & SE
EV_MPI = 7.36  #           EV max Power input for each time period
SE_MPO = 7.36  #           EVSE max Power output for each time period
EX_MPO = n * r * SE_MPO  # enexis max Power output for each time period (constant)

# The array's for all EV's & SE together and the resulting minimum.
EV = np.random.choice([EV_MPI, EV_MPI / 2, 3.33, 3.33 / 2], size=(m, n))
SE = np.random.choice([SE_MPO], size=(m, n))
EVSE = np.minimum(EV, SE)

if print_EVSE_power:
    print(f"\nEVSE:\n {EVSE}\n")

##### TGC: Central Controller

In [12]:
# Create the weights w[j] for the parking
# pt = np.array([40, 10, 20, 10, 10]) / 60
pt = np.random.choice([40, 10, 20, 10, 10], size=(n))/60
if print_parking_time:
    print(f"\nparking time:\n {pt}\n")
w = pt / np.sum(pt)

# calculate the maximum charge possible for each session
session_max_charge = np.sum(pt * EVSE, axis=1)
if print_session_max_charge:
    print(f"\nsession_max_charge:\n {session_max_charge}\n")

# desired charge for each session, max battery capacity = set to 70 kWh
desired_charge = np.random.uniform(1, 20, size=m)
if print_desired_charge:
    print(f"\ndesired_charge:\n {desired_charge}\n")

# realistic charge for each session
realistic_charge = np.minimum(session_max_charge, desired_charge)
if print_realistic_charge:
    print(f"\nrealistic_charge:\n {realistic_charge}\n")

# energy price in euro/kWh for each time period
# EnergyPrice = np.array([0.04, 0.08, 0.31, 0.28, 0.12])
EnergyPrice = np.random.uniform(-0.1, 0.4, size=n)
Energy_Cost = EnergyPrice * pt
if print_energy_price:
    print(f"\nenergy_price:\n {energy_price}\n")


parking time:
 [0.33333333 0.16666667 0.16666667 0.33333333 0.16666667 0.16666667
 0.16666667 0.16666667 0.16666667 0.16666667]


session_max_charge:
 [8.0175     6.06083333 6.95166667 7.79833333 6.0025     7.74
 5.66666667 7.565      6.67416667 6.455      8.0175     6.5575
 8.295      7.01       9.08333333 7.74       8.295      9.18583333
 6.06083333 6.7325    ]


desired_charge:
 [19.56996841  7.7196914   7.8849312   5.75204803  8.43535551 16.61971531
 17.04128207  2.49023486 18.29656368 17.23078773  9.55176247  9.92364709
  8.94179324 15.48587343  8.7591428  13.75140289  9.38334715  5.33678072
  9.96125643  9.74348443]


realistic_charge:
 [8.0175     6.06083333 6.95166667 5.75204803 6.0025     7.74
 5.66666667 2.49023486 6.67416667 6.455      8.0175     6.5575
 8.295      7.01       8.7591428  7.74       8.295      5.33678072
 6.06083333 6.7325    ]



In [13]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# energy price per kWh for each time period
energy_price = {(j + 1): EnergyPrice[j] for j in range(len(EnergyPrice))}
model.energy_price = Param(model.J, initialize=energy_price)

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j] for i in range(len(EVSE)) for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)

# realistic charge for each session
realistic_charge_kwh = {
    (i + 1): realistic_charge[i] for i in range(len(realistic_charge))
}
model.realistic_charge = Param(model.I, initialize=realistic_charge_kwh)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return (
        alpha
        * sum(
            w[j - 1] * (1 - sum(model.x[i, j] for i in model.I) / model.enexis[j])
            for j in model.J
        )
        + beta
        * sum(
            (
                1
                - sum(pt[j - 1] * model.x[i, j] for j in model.J)
                / model.realistic_charge[i]
            )
            for i in model.I
        )
        / m
        + (gamma / sum(Energy_Cost[j - 1] * model.enexis[j] for j in model.J))
        * sum(
            (Energy_Cost[j - 1] * sum(model.x[i, j] for i in model.I)) for j in model.J
        )
    )


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def ex_grid_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= session max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# is final charge <= realistic charge <= desired charge
def deschrg_constraint_rule(model, i):
    return sum(pt[j - 1] * model.x[i, j] for j in model.J) <= model.realistic_charge[i]


# the next line creates one constraint for each member of the set model.J
model.Ex_Grid_Constraint = Constraint(model.J, rule=ex_grid_constraint_rule)
model.DesChrg_Constraint = Constraint(model.I, rule=deschrg_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)

# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory(solver)

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# outcome of solver model
if print_solver_outcome:
    print(f"\nsolver outcome:\n {tgc.pprint()}\n")


# print the power per EVSE per time period
def EVSE_print():
    for i in range(1, m + 1):
        print(
            f"EVSE {str(i).zfill(2)}: DC: {round(desired_charge[i-1], 2)} kWh\tRC: {round(realistic_charge[i-1], 2)} kWh\tFC: {round(sum(pt[x-1] * tgc.x[i, x].value for x in range(1, n+1)), 2)} kWh\tPwr: {[round(tgc.x[i, x].value, 2) for x in range(1, n+1)]} kW"
        )


EVSE_print()

# print the objective outcome
print("\n")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))

EVSE 01: DC: 19.57 kWh	RC: 8.02 kWh	FC: 0.0 kWh	Pwr: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] kW
EVSE 02: DC: 7.72 kWh	RC: 6.06 kWh	FC: 4.67 kWh	Pwr: [1.67, 3.68, 7.36, 0.0, 0.0, 1.67, 3.68, 3.33, 1.67, 3.33] kW
EVSE 03: DC: 7.88 kWh	RC: 6.95 kWh	FC: 1.89 kWh	Pwr: [0.0, 1.67, 0.0, 0.0, 0.0, 0.0, 2.93, 1.67, 3.33, 1.75] kW
EVSE 04: DC: 5.75 kWh	RC: 5.75 kWh	FC: 4.12 kWh	Pwr: [1.67, 3.68, 3.68, 0.0, 0.0, 1.67, 1.67, 3.68, 3.68, 3.33] kW
EVSE 05: DC: 8.44 kWh	RC: 6.0 kWh	FC: 4.22 kWh	Pwr: [3.33, 1.67, 3.68, 0.0, 0.0, 3.33, 1.67, 3.33, 3.33, 1.67] kW
EVSE 06: DC: 16.62 kWh	RC: 7.74 kWh	FC: 0.19 kWh	Pwr: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.14, 0.0] kW
EVSE 07: DC: 17.04 kWh	RC: 5.67 kWh	FC: 4.5 kWh	Pwr: [3.33, 3.68, 3.33, 0.0, 0.0, 3.33, 3.33, 3.33, 1.67, 1.67] kW
EVSE 08: DC: 2.49 kWh	RC: 2.49 kWh	FC: 2.49 kWh	Pwr: [0.0, 3.07, 0.0, 0.0, 0.0, 0.0, 5.22, 3.33, 3.33, 0.0] kW
EVSE 09: DC: 18.3 kWh	RC: 6.67 kWh	FC: 4.63 kWh	Pwr: [0.0, 3.68, 1.67, 0.0, 0.0, 6.75, 3.33, 1.67, 3.3